<a href="https://colab.research.google.com/github/tmendonca28/DLNID/blob/master/RNN_with_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pdb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from statistics import mean
import pandas as pd
import numpy as np
import torch.optim as optim

In [19]:
train_data = pd.read_csv("drive/My Drive/Masters/Project/DLNID/Datasets/KDDTrainRenameNominalValuesNormalizedOHE.csv", header=0, low_memory=False, dtype = np.float32)
test_data = pd.read_csv("drive/My Drive/Masters/Project/DLNID/Datasets/KDDTestRenameNominalValuesNormalizedOHE.csv", header=0, low_memory=False, dtype = np.float32)
test21_data = pd.read_csv("drive/My Drive/Masters/Project/DLNID/Datasets/KDDTest-21RenameNominalValuesNormalizedOHE.csv", header=0, low_memory=False, dtype = np.float32)
print(len(train_data))
print(len(test_data))
print(len(test21_data))
print(test_data)

125973
22544
11850
       duration  protocol_type=1  ...  dst_host_srv_rerror_rate  class
0      0.000000              1.0  ...                      1.00    1.0
1      0.000000              1.0  ...                      1.00    1.0
2      0.000035              1.0  ...                      0.00    0.0
3      0.000000              0.0  ...                      0.00    1.0
4      0.000017              1.0  ...                      0.71    1.0
5      0.000000              1.0  ...                      0.00    0.0
6      0.000000              1.0  ...                      0.04    0.0
7      0.000000              1.0  ...                      0.02    1.0
8      0.000000              1.0  ...                      0.00    0.0
9      0.000000              1.0  ...                      0.00    1.0
10     0.000000              1.0  ...                      0.32    1.0
11     0.000000              1.0  ...                      0.00    0.0
12     0.000000              1.0  ...                     

In [20]:
targets_pd = train_data.iloc[0:, 122]
features_pd = train_data.iloc[0:, 0:122]

test_targets_pd = test_data.iloc[0:, 122]
test_features_pd = test_data.iloc[0:, 0:122]

test21_targets_pd = test21_data.iloc[0:, 122]
test21_features_pd = test21_data.iloc[0:, 0:122]

# create feature and targets tensor for train set. As you remember we need variable to accumulate gradients. Therefore first we create tensor, then we will create variable
featuresTrain = torch.FloatTensor(features_pd.values)
targetsTrain = torch.LongTensor(targets_pd.values)

featuresTest = torch.FloatTensor(test_features_pd.values)
targetsTest = torch.LongTensor(test_targets_pd.values)

featuresTest21 = torch.FloatTensor(test21_features_pd.values)
targetsTest21 = torch.LongTensor(test21_targets_pd.values)

# batch_size and epoch
batch_size = 1000
num_epochs = 100

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(featuresTrain,targetsTrain)
test = torch.utils.data.TensorDataset(featuresTest, targetsTest)
test21 = torch.utils.data.TensorDataset(featuresTest21, targetsTest21)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False, drop_last=True)
test21_loader = torch.utils.data.DataLoader(test21, batch_size = batch_size, shuffle = False, drop_last=True)
print(len(train))
print(featuresTrain.size())
print(targetsTrain.size())
print(featuresTest.size())
print(targetsTest.size())
print(featuresTest21.size())
print(targetsTest21.size())
print(len(test))
print(len(train_loader))
print(len(test_loader))
print(len(test21_loader))

125973
torch.Size([125973, 122])
torch.Size([125973])
torch.Size([22544, 122])
torch.Size([22544])
torch.Size([11850, 122])
torch.Size([11850])
22544
125
22
11


In [21]:
import argparse
parser = argparse.ArgumentParser(description='Network Intrusion Classifier')
parser.add_argument('--lr', action='store', type=float, help='use different versions of network', default=1e-1)
parser.add_argument('--epochs', action='store', type=int, help='use different versions of network', default=num_epochs)
opts = parser.parse_args('')
print(opts)

Namespace(epochs=100, lr=0.1)


In [0]:
# Defining the RNN Model
class RNNModel(nn.Module):
  def __init__(self ):
    super(RNNModel, self).__init__()
    self.rnn = nn.RNN(122, 80, 1, batch_first=True, nonlinearity='relu')
    self.fc = nn.Linear(80, 2)
    self.layer_dim = 1
    self.hidden_dim = 80
    
    
  def forward(self, x):
    h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
    out, hn = self.rnn(x, h0.detach())
    out = self.fc(out[:, -1, :])
    return out

In [0]:
def evaluate_network_performance(predictions, truth):
  # given a list with predicted and correct values
  # calculate precision, recall and f-score
  cm = get_confusion_matrix(predictions, truth)
  TP = cm[0][0]
  FP = cm[0][1]
  FN = cm[1][0]
  TN = cm[1][1]

  precision = TP/(TP+FP)
  recall = TP/(TP+FN)
  classification_accuracy = (TP+TN) / (TP + TN + FP + FN)
  f_score = 2 * ((precision*recall)/(precision + recall))

  return precision, recall, f_score, classification_accuracy

In [0]:
def get_confusion_matrix(preds, truth):
    preds = torch.stack(preds).flatten()
    truth = torch.stack(truth).flatten()
    K = len(np.unique(truth)) # Number of classes
    result = np.zeros((K, K))
    for i in range(len(truth)):
        result[preds[i]][truth[i]] += 1
    confusion_matrix = result
    return confusion_matrix

In [0]:
def test_network(model, test_set, targets):
  # given a dataset and network, compute accuracy
  
  criterion = nn.CrossEntropyLoss()
  
  test_loss = 0
  predictions = []
  labels = []
  
  with torch.no_grad():
    for i, (images, label) in enumerate(test_set):
      model.eval()
      
      images = Variable(images.view(-1, 1, 122))
      output = model(images)
      predicted_index = torch.max(output.data, 1)[1]
      predictions.append(predicted_index)
      labels.append(label)
      
      test_loss = criterion(output, label).item()
    precision, recall, f_score, classification_accuracy = evaluate_network_performance(predictions, labels)

  return precision, recall, f_score, classification_accuracy

In [0]:
def train_network(model, train_dataset, test_dataset):
  optimizer = optim.Adam(model.parameters(), lr=opts.lr)
  criterion = nn.CrossEntropyLoss()
  
  num_training_epochs = opts.epochs
  
  for e in range(num_training_epochs):
    print("Running Epoch: ", e, " ....")
    running_loss = 0
    
    for i,(images, labels) in enumerate(train_dataset):
      model.train()
      optimizer.zero_grad()
      labels = Variable(labels)
      images = Variable(images.view(-1, 1, 122))  
  
      output = model(images)
      loss = criterion(output, labels)
      
      running_loss += loss.item()
      
      loss.backward()
      optimizer.step()
      
    epoch_loss = running_loss/(len(train_dataset))
    
    print('Iteration ', e, 'loss ', epoch_loss)
    print('          PREC | REC | F1 | ACC')
    precision, recall, f_score, classification_accuracy = test_network(model, train_dataset, targetsTrain)
    print('test set',"{0:.2f} | ".format(precision),"{0:.2f} | ".format(recall),"{0:.2f} | ".format(f_score),"{0:.2f}".format(classification_accuracy))

    precision, recall, f_score, classification_accuracy = test_network(model, test_dataset, targetsTest)
    print('train set',"{0:.2f} | ".format(precision),"{0:.2f} | ".format(recall),"{0:.2f} | ".format(f_score),"{0:.2f}".format(classification_accuracy))
    print()

  return model

In [0]:
def main():
  model = RNNModel()
  print(model)
  print('Started training network......')
  model = train_network(model, train_loader, test_loader)
  
  print('Final Testing.......')
  precision, recall, f_score, classification_accuracy = test_network(model, test_loader, targetsTest)
  print('test set',"{0:.2f}".format(precision),"{0:.2f}".format(recall),"{0:.2f}".format(f_score),"{0:.2f}".format(classification_accuracy))
  print()

In [28]:
main()

RNNModel(
  (rnn): RNN(122, 80, batch_first=True)
  (fc): Linear(in_features=80, out_features=2, bias=True)
)
Started training network......
Running Epoch:  0  ....
Iteration  0 loss  0.10034591399133205
          PREC | REC | F1 | ACC
test set   0.99 |  0.98 |  0.98 |  0.98
train set 0.66 |  0.92 |  0.77 |  0.77

Running Epoch:  1  ....
Iteration  1 loss  0.037485645949840544
          PREC | REC | F1 | ACC
test set   0.99 |  0.99 |  0.99 |  0.99
train set 0.69 |  0.92 |  0.79 |  0.78

Running Epoch:  2  ....
Iteration  2 loss  0.028136018231511115
          PREC | REC | F1 | ACC
test set   0.98 |  1.00 |  0.99 |  0.99
train set 0.68 |  0.97 |  0.80 |  0.79

Running Epoch:  3  ....
Iteration  3 loss  0.02232839171588421
          PREC | REC | F1 | ACC
test set   0.99 |  0.99 |  0.99 |  0.99
train set 0.68 |  0.97 |  0.80 |  0.79

Running Epoch:  4  ....
Iteration  4 loss  0.020518202863633633
          PREC | REC | F1 | ACC
test set   0.99 |  0.99 |  0.99 |  0.99
train set 0.69 |  0.9

KeyboardInterrupt: ignored